# Estimate Ingredient Quantites from Nutrient Info

Given a food product with an FDA label listing $d$ nutrient quantities (e.g. Calories, carbohydrates, etc.) and $p$ ingredients ordered by weight, we calculate the quantity of each ingredient in the product.

We take a constrained optimization approach, seeking the optimal solution $x$ to the linear system $Ax=b$, where $A$ is a matrix describing the nutrient contents of the ingredients, $b$ is a vector listing the total nutrient contents of the products, and $x$ is the quantity of each ingredient.

The system is subject to the following constraints:
- All ingredient quantities are positive ($x>0$)
- The ingredient quantities are ordered ($x_{i+1}>x_i$) as given in the ingredients list.

**NOTE:** When the number of ingredients is greater than the number of nutrients on the FDA label (when $p>d$), there is no unique solution, and further analysis is required to construct bounds for the nutrient quantities. 

## Imports

In [1]:
import re

import numpy as np
import scipy.optimize as optim

In [6]:
import qgrid
import pandas as pd

In [7]:
import ipywidgets as ipw

In [8]:
from IPython.display import display, Markdown

## Synthesize Data

In [2]:
def gen_data(d, p):
    """
    d = Number of FDA requirements (known)
    p = Number of ingredients
    """
    
    # Coefficient matrix:
    # FDA nutrient info for each ingredient
    # - each column is an ingredient
    # - FDA nutrient varies over rows
    # - Sorted in order of increasing quantity (first is smallest)
    A = np.random.rand(d, p)

    # Quantities of each ingredient, sorted in increasing order.
    # (correct solution)
    x = np.sort(np.random.rand(p))

    # Sum FDA nutrients of the product
    b = A@x
    
    return A, x, b

## Solve optimization problem

In [3]:
def estim_quantities(A, b, tol=None):
    # Objective function
    def obj(x):
        return np.sum((A@x-b)**2)

    # Ordering constraint matrix
    # each row: c_i^T * x >= 0
    C = np.zeros([p, p])
    # Require that each value is larger than the previous
    for i in range(p):
        for j in range(p):
            if i == j:
                C[i,j] = 1
            elif j == i-1:
                C[i,j] = -1

    # non-negative constraint (0 <= c_i^T *x <= inf)
    lin_const = optim.LinearConstraint(C, 0, np.inf, keep_feasible=False)
    
    # Solve!
    result = optim.minimize(obj, np.zeros(p), method="COBYLA", constraints=lin_const, tol=tol)

    return result

## Apply some noise to simulate real data

In [4]:
def apply_noise(A, z):
    return A * (1.0 + z * 2*(np.random.rand(*np.shape(A))-0.5))

## Test it out!

In [5]:
# Number of FDA requirements (known)
d = 10
# Number of ingredients
p = 5
# Relative noise to apply to A (reported FDA vals. for ingredients)
zA = 0.05
# Relative noise to apply to b (reported FDA vals. for product)
zb = 0.05

# Generate data & apply noise
A, x, b = gen_data(d, p)
A_noise = apply_noise(A, zA)
b_noise = apply_noise(b, zb)

#print("x = {}".format(x))
print()

print("Without noise:")
res = estim_quantities(A, b)
x_star = res.x
#print("x* = {}".format(x_star))
print("rel. err. = {:.2e}".format(np.linalg.norm(x-x_star)/np.linalg.norm(x)))
print()

print("With noise (zA={:.2e}, zb={:.2e}):".format(zA, zb))
res = estim_quantities(A_noise, b_noise)
x_star = res.x
#print("x_n* = {}".format(x_star))
print("rel. err. = {:.2e}".format(np.linalg.norm(x-x_star)/np.linalg.norm(x)))


Without noise:
rel. err. = 2.69e-03

With noise (zA=5.00e-02, zb=5.00e-02):
rel. err. = 1.47e-01


## FDA Nutrient Table

In [112]:
def disp_nutrient_table(nutrients, vals):
    display(Markdown(
        "| Nutrient | Value |\n|---|---|\n"
        + '\n'.join([
            "| {} | {:.2f} |".format(nutrient, val)
            for nutrient, val in zip(nutrients, vals)
        ])
    ))

    
def gen_fda_label(ingredients, nutrients, vals):
    inner_tab = ipw.Output()
    with inner_tab:
        disp_table(nutrients, vals)
    return ipw.VBox(
        [
            ipw.HTML("<h2>Nutrition Facts</h2>",),
            inner_tab,
            ipw.Box(layout=ipw.Layout(height='5px')),
            ipw.HTML("<b>Ingredients:</b> {}".format(
                ', '.join(ingredients)
            ))
            
        ],
        layout=ipw.Layout(
            border='1px solid black',
            width='200px',
            align_items='center',
        )
    )

In [113]:
ingredients = [
    'tomato',
    'sugar',
    'onion',
    'garlic',
    'squash'
]

In [116]:
gen_fda_label(ingredients, nutrients, vals)

## Ingredients Nutrition Data

In [177]:
def parse_fda_db_csv(filename):
    nutrient_name_map = {
        'Calories': 'Energy',
        'Total Fat': 'Total lipid (fat)',
        'Cholesterol': 'Cholesterol',
        'Sodium': 'Sodium, Na',
        'Total Carbohydrate': 'Carbohydrate, by difference',
        'Dietary Fiber': 'Fiber, total dietary',
        'Total Sugars': 'Sugars, total',
        'Protein': 'Protein'
    }
    
    df = pd.read_csv(filename, skiprows=4, encoding='latin1')
    nut_dict = {}
    for name_common, name_db in nutrient_name_map.items():
        filtered_df = df[df['Nutrient']==name_db]
        if len(filtered_df) > 0:
            # Convert from value per 100g to value per 1g
            nut_val = filtered_df['1Value per 100 g'].iloc[0] / 100
            nut_units = filtered_df['Unit'].iloc[0]
            name_with_units = '{} ({}/g)'.format(name_common, nut_units)
            nut_dict[name_with_units] = nut_val
    return nut_dict

In [178]:
parse_fda_db_csv('garlic.csv')

{'Calories (kcal/g)': 1.49,
 'Total Fat (g/g)': 0.005,
 'Cholesterol (mg/g)': 0.0,
 'Sodium (mg/g)': 0.17,
 'Total Carbohydrate (g/g)': 0.3306,
 'Dietary Fiber (g/g)': 0.021,
 'Total Sugars (g/g)': 0.01,
 'Protein (g/g)': 0.0636}

In [179]:
ingredients = [
    'Broccoli',
    'Carrots',
    'Onions',
    'Garlic',
    'Tomatoes'
]

In [184]:
nutrient_info = {
    ingredient: parse_fda_db_csv('ingredient_data/{}.csv'.format(ingredient.lower()))
    for ingredient in ingredients
}

In [185]:
nutrient_info_df = pd.DataFrame(nutrient_info)
nutrient_info_df

,Broccoli,Carrots,Onions,Garlic,Tomatoes
Calories (kcal/g),0.3400,0.3400,0.4000,1.4900,0.1800
Cholesterol (mg/g),0.0000,0.0000,0.0000,0.0000,0.0000
Dietary Fiber (g/g),0.0260,0.0260,0.0170,0.0210,0.0120
Protein (g/g),0.0282,0.0282,0.0110,0.0636,0.0088
Sodium (mg/g),0.3300,0.3300,0.0400,0.1700,0.0500
Total Carbohydrate (g/g),0.0664,0.0664,0.0934,0.3306,0.0389
Total Fat (g/g),0.0037,0.0037,0.0010,0.0050,0.0020
Total Sugars (g/g),0.0170,0.0170,0.0424,0.0100,0.0263


In [222]:
example_recipe = {
    'Tomatoes': 100,
    'Onions': 50,
    'Garlic': 30,
    'Broccoli': 50,
    'Carrots': 60
}

In [376]:
# String manipulation helpers
def str_density_to_total(nut_str):
    return re.sub(r'\((.*)\/g\)', r'(\1)', nut_str)

def str_total_to_density(nut_str):
    return re.sub(r'\((.*)\)', r'(\1/g)', nut_str)

In [381]:
def calculate_nutrition_facts(recipe, nutrient_info):
    nutrition_facts = {
        str_density_to_total(nutrient): 0.0
        for nutrient in nutrient_info[list(nutrient_info.keys())[0]].keys()
    }
    
    # quantities in grams
    for ingredient, quantity in recipe.items():
        for nutrient, value in nutrient_info[ingredient].items():
            total_nutrient = str_density_to_total(nutrient)
            nutrition_facts[total_nutrient] += quantity * value
            
    return nutrition_facts     

In [382]:
def sort_ingredient_list(recipe):
    # Return list of ingredients in decreasing order of quantity in grams
    return [x[0] for x in reversed(sorted(recipe.items(), key=lambda x: x[1]))]

In [383]:
ingredient_list = sort_ingredient_list(example_recipe)
ingredient_list

['Tomatoes', 'Carrots', 'Broccoli', 'Onions', 'Garlic']

In [384]:
nutrition_facts = calculate_nutrition_facts(example_recipe, nutrient_info)
nutrition_facts

{'Calories (kcal)': 120.10000000000001,
 'Total Fat (g)': 0.8069999999999999,
 'Cholesterol (mg)': 0.0,
 'Sodium (mg)': 48.400000000000006,
 'Total Carbohydrate (g)': 25.782000000000004,
 'Dietary Fiber (g)': 5.539999999999999,
 'Total Sugars (g)': 6.92,
 'Protein (g)': 6.44}

In [385]:
gen_fda_label(ingredient_list, nutrition_facts.keys(), nutrition_facts.values())

In [535]:
def gen_fda_label_from_recipe(recipe, nutrient_info):
    ingredient_list = sort_ingredient_list(recipe)
    nutrition_facts = calculate_nutrition_facts(recipe, nutrient_info)
    return gen_fda_label(ingredient_list, nutrition_facts.keys(), nutrition_facts.values())

In [536]:
gen_fda_label_from_recipe(example_recipe, nutrient_info)

In [532]:
def recipe_wrapper(nutrient_info, **recipe):
    return disp_fda_label_from_recipe(recipe, nutrient_info)

In [441]:
def solve_recipe(recipe, nutrient_info):
    nutrition_facts = calculate_nutrition_facts(recipe, nutrient_info)
    # Sorted in decreasing order - have to reverse
    sorted_ingredients = sort_ingredient_list(recipe)[::-1]

    # Convert to correctly sorted arrays
    df = pd.DataFrame(nutrient_info)
    A = np.array(df[sorted_ingredients], dtype=float)

    # Save order and values of nutrients
    nutrients = list(map(str_density_to_total, df.index))
    nutrient_vals = [nutrition_facts[nutrient] for nutrient in nutrients]
    b = np.array(nutrient_vals, dtype=float)

    # Solve
    result = estim_quantities(A, b)
    x = result.x
    
    # Convert back to recipe
    estim_recipe = dict(zip(sorted_ingredients, x))
    
    return estim_recipe

In [592]:
def gen_recipe_comparison(recipe, estim_recipe):
    ingredients = list(recipe.keys())
    x_true = np.array([recipe[ing] for ing in ingredients])
    x_estim = np.array([estim_recipe[ing] for ing in ingredients])
    individual_abs_errs = np.abs(x_true - x_estim)
    individual_rel_errs = individual_abs_errs / x_true
    
    global_rel_err = np.linalg.norm(x_true-x_estim) / np.linalg.norm(x_true)
    
    data = np.array([x_true, x_estim, individual_abs_errs, 100*individual_rel_errs]).T
    cols = ['True Recipe (g)', 'Estim. Recipe (g)', 'Abs. Error (g)', 'Rel. Error (%)']
    df = pd.DataFrame(data, index=ingredients, columns=cols)
    
    out = ipw.Output()
    with out:
        display(HTML("<h3>Recipe Estimation Results</h3>"))
        display(ipw.Box(layout=ipw.Layout(height='5px')))
        display(HTML(df.to_html(float_format=lambda s: '{:.2f}'.format(s))))
        print("Global Relative Error: {:.2e}".format(global_rel_err))
    return out

In [593]:
gen_recipe_comparison(example_recipe, estim_recipe)

Output()

In [594]:
def full_widget_output(recipe, nutrient_info):
    fda_label = gen_fda_label_from_recipe(recipe, nutrient_info)
    estim_recipe = solve_recipe(recipe, nutrient_info)
    results_table = gen_recipe_comparison(recipe, estim_recipe)
    
    widget = ipw.HBox(
        [
            fda_label,
            results_table
        ],
        layout=ipw.Layout(
            width='800px',
            justify_content='space-between'
        )
    )
    
    return widget
    

In [595]:
full_widget_output(example_recipe, nutrient_info)

In [596]:
def widget_wrapper(**recipe):
    display(full_widget_output(recipe, nutrient_info))

In [597]:
inter_out = ipw.interactive_output(widget_wrapper, sliders)

In [598]:
ipw.VBox([
    slider_box,
    inter_out
])

In [599]:
class RecipeWidget(ipw.VBox):
    def __init__(self, recipe=None):
        if recipe is None:
            self.recipe = example_recipe
        else:
            self.recipe = recipe
            
        self.do_calculations()
        self.init_outputs()
        self.init_layout()
    
    def do_calculations(self):
        self.estim_recipe = solve_recipe(self.recipe, nutrient_info)
    
    def init_sliders(self):
        ingredients = list(self.recipe.keys())
        self.sliders = [
            ipw.IntSlider(
                description=ingredient,
                value=self.recipe[ingredient],
                min=0, 
                max=150
            )
            for ingredient in ingredients
        ]
        self.slider_box = ipw.VBox(self.sliders)
    
    def init_outputs(self):
        self.init_sliders()
        
        self.fda_label = gen_fda_label_from_recipe(self.recipe, nutrient_info)
        self.results_table = gen_recipe_comparison(self.recipe, self.estim_recipe)
    
    def init_layout(self):
        super().__init__([
            ipw.HBox(
                [
                    self.fda_label,
                    ipw.VBox([
                        self.slider_box,
                        self.results_table
                    ])
                ],
                layout=ipw.Layout(
                    width='800px',
                    justify_content='space-between'
                )
            )
        ])

In [600]:
RecipeWidget()

RecipeWidget(children=(HBox(children=(VBox(children=(HTML(value='<h2>Nutrition Facts</h2>'), Output(), Box(lay…